In [23]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [33]:
%cd "/content/drive/My Drive/Projects/quantum_image_classifier/PennyLane/Data Reuploading Classifier"

/content/drive/My Drive/Projects/quantum_image_classifier/PennyLane/Data Reuploading Classifier


In [1]:
!pip install pennylane
from IPython.display import clear_output
clear_output()

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

In [2]:
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np

# Loading Raw Data

In [3]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[:, 0:27, 0:27]
x_test = x_test[:, 0:27, 0:27]

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
x_train_flatten = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])/255.0
x_test_flatten = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])/255.0

In [5]:
print(x_train_flatten.shape, y_train.shape)
print(x_test_flatten.shape, y_test.shape)

(60000, 729) (60000,)
(10000, 729) (10000,)


In [6]:
x_train_0 = x_train_flatten[y_train == 0]
x_train_1 = x_train_flatten[y_train == 1]
x_train_2 = x_train_flatten[y_train == 2]
x_train_3 = x_train_flatten[y_train == 3]
x_train_4 = x_train_flatten[y_train == 4]
x_train_5 = x_train_flatten[y_train == 5]
x_train_6 = x_train_flatten[y_train == 6]
x_train_7 = x_train_flatten[y_train == 7]
x_train_8 = x_train_flatten[y_train == 8]
x_train_9 = x_train_flatten[y_train == 9]

x_train_list = [x_train_0, x_train_1, x_train_2, x_train_3, x_train_4, x_train_5, x_train_6, x_train_7, x_train_8, x_train_9]

print(x_train_0.shape)
print(x_train_1.shape)
print(x_train_2.shape)
print(x_train_3.shape)
print(x_train_4.shape)
print(x_train_5.shape)
print(x_train_6.shape)
print(x_train_7.shape)
print(x_train_8.shape)
print(x_train_9.shape)

(5923, 729)
(6742, 729)
(5958, 729)
(6131, 729)
(5842, 729)
(5421, 729)
(5918, 729)
(6265, 729)
(5851, 729)
(5949, 729)


In [7]:
x_test_0 = x_test_flatten[y_test == 0]
x_test_1 = x_test_flatten[y_test == 1]
x_test_2 = x_test_flatten[y_test == 2]
x_test_3 = x_test_flatten[y_test == 3]
x_test_4 = x_test_flatten[y_test == 4]
x_test_5 = x_test_flatten[y_test == 5]
x_test_6 = x_test_flatten[y_test == 6]
x_test_7 = x_test_flatten[y_test == 7]
x_test_8 = x_test_flatten[y_test == 8]
x_test_9 = x_test_flatten[y_test == 9]

x_test_list = [x_test_0, x_test_1, x_test_2, x_test_3, x_test_4, x_test_5, x_test_6, x_test_7, x_test_8, x_test_9]

print(x_test_0.shape)
print(x_test_1.shape)
print(x_test_2.shape)
print(x_test_3.shape)
print(x_test_4.shape)
print(x_test_5.shape)
print(x_test_6.shape)
print(x_test_7.shape)
print(x_test_8.shape)
print(x_test_9.shape)

(980, 729)
(1135, 729)
(1032, 729)
(1010, 729)
(982, 729)
(892, 729)
(958, 729)
(1028, 729)
(974, 729)
(1009, 729)


# Selecting the dataset

Output: X_train, Y_train, X_test, Y_test

In [8]:
X_train = np.concatenate((x_train_list[0][:200, :], x_train_list[1][:200, :]), axis=0)
Y_train = np.zeros((X_train.shape[0],), dtype=int)
Y_train[200:] += 1

X_train.shape, Y_train.shape

((400, 729), (400,))

In [9]:
X_test = np.concatenate((x_test_list[0][:500, :], x_test_list[1][:500, :]), axis=0)
Y_test = np.zeros((X_test.shape[0],), dtype=int)
Y_test[500:] += 1

X_test.shape, Y_test.shape

((1000, 729), (1000,))

# Dataset Preprocessing

In [10]:
X_train = X_train.reshape(X_train.shape[0], 27, 27)
X_test = X_test.reshape(X_test.shape[0], 27, 27)

X_train.shape, X_test.shape

((400, 27, 27), (1000, 27, 27))

# Quantum

In [11]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

qml.enable_tape()

from tensorflow.keras.utils import to_categorical

# Set a random seed
np.random.seed(2020)

In [12]:
# Define output labels as quantum state vectors
def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T


label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = [label_0, label_1]

In [13]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs = [x, y]
        x (array[float]): 1-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]


In [14]:
dev_conv = qml.device("default.qubit", wires=3)


@qml.qnode(dev_conv)
def q_conv(conv_params, inputs):
    """A variational quantum circuit representing the Universal classifier + Conv.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    # layer iteration
    for l in range(len(conv_params[0])):
        # qubit iteration
        for q in range(3):
            qml.Rot(*(conv_params[0][l][3*q:3*(q+1)] * inputs[q, 0:3] + conv_params[1][l][3*q:3*(q+1)]), wires=q)

    return [qml.expval(qml.PauliZ(j)) for j in range(3)]

In [15]:
from keras import backend as K

# Addition Custom Layer
def add_matrix(x):
    return K.sum(x, axis=1, keepdims=True)
    
addition_layer = tf.keras.layers.Lambda(add_matrix, output_shape=(1,))


# Alpha Custom Layer
class class_weights(tf.keras.layers.Layer):
    def __init__(self):
        super(class_weights, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(1, 2), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):
        return (inputs * self.w)

In [16]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(3), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = addition_layer(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(3), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = addition_layer(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)


# Quantum FC Layer, trainable params = 18+2, output size = 2
num_fc_layer = 1
q_fc_layer = qml.qnn.KerasLayer(q_fc, {"params": (2, num_fc_layer, 9)}, output_dim=2, name='Quantum_FC_Layer')(reshape_layer_3)
# Alpha Layer, trainable params = 2
class_weights_layer = class_weights()(q_fc_layer)


model = tf.keras.Model(inputs=X, outputs=class_weights_layer)

In [17]:
model(X_train[0:32, :, :])

<tf.Tensor: shape=(32, 2), dtype=float32, numpy=
array([[-0.05036977, -0.00690893],
       [-0.0504736 , -0.00673647],
       [-0.05039774, -0.00686247],
       [-0.05041588, -0.00683234],
       [-0.05038233, -0.00688807],
       [-0.05042683, -0.00681415],
       [-0.05041436, -0.00683486],
       [-0.05035771, -0.00692895],
       [-0.05040242, -0.0068547 ],
       [-0.05040852, -0.00684457],
       [-0.05039135, -0.00687309],
       [-0.05029934, -0.0070259 ],
       [-0.05036765, -0.00691245],
       [-0.05034237, -0.00695443],
       [-0.0504345 , -0.00680142],
       [-0.05039123, -0.00687328],
       [-0.05043   , -0.00680889],
       [-0.05034036, -0.00695776],
       [-0.05041303, -0.00683706],
       [-0.05037367, -0.00690245],
       [-0.05038919, -0.00687666],
       [-0.05032244, -0.00698752],
       [-0.05030955, -0.00700894],
       [-0.0504129 , -0.00683728],
       [-0.05041363, -0.00683607],
       [-0.05035234, -0.00693786],
       [-0.05046547, -0.00674997],
      

In [18]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(opt, loss="mse", metrics=["accuracy"])

In [19]:
model.fit(X_train, to_categorical(Y_train), epochs=6, batch_size=32, validation_data=(X_test, to_categorical(Y_test)), verbose=1)

Epoch 1/6
13/13 [==============================] - 3993s 314s/step - loss: 0.3865 - accuracy: 0.6081 - val_loss: 0.0379 - val_accuracy: 0.9620
Epoch 2/6
13/13 [==============================] - 3962s 312s/step - loss: 0.0578 - accuracy: 0.9456 - val_loss: 0.0411 - val_accuracy: 0.9660
Epoch 3/6
13/13 [==============================] - 3978s 313s/step - loss: 0.0339 - accuracy: 0.9736 - val_loss: 0.0286 - val_accuracy: 0.9830
Epoch 4/6
13/13 [==============================] - 4004s 315s/step - loss: 0.0225 - accuracy: 0.9751 - val_loss: 0.0189 - val_accuracy: 0.9830
Epoch 5/6
13/13 [==============================] - 4004s 315s/step - loss: 0.0193 - accuracy: 0.9832 - val_loss: 0.0136 - val_accuracy: 0.9920
Epoch 6/6
13/13 [==============================] - 4011s 316s/step - loss: 0.0121 - accuracy: 0.9887 - val_loss: 0.0189 - val_accuracy: 0.9900


In [36]:
model.get_weights()

[array([[[ 0.5333041 ,  0.7510351 ,  0.43303   , -0.40551516,
           1.6285696 ,  0.49228728, -0.15253374,  1.6689872 ,
          -0.22357488]],
 
        [[-0.2997148 ,  0.7048321 ,  0.4084043 , -0.18627408,
           1.0520505 , -0.00270981, -0.13496283,  1.0413212 ,
          -0.02724391]]], dtype=float32),
 array([[[ 0.3918922 , -0.11520633, -0.03320652,  0.43418598,
          -0.5833789 ,  0.39737093, -0.5246267 , -0.9033116 ,
           0.48647797]],
 
        [[-0.07068413, -0.3555113 , -0.41528818,  0.4065122 ,
          -0.15927339,  0.1401099 ,  0.4446873 , -0.7067284 ,
          -0.545056  ]]], dtype=float32),
 array([[[-3.8316455e-01, -3.1901807e-02, -2.4000183e-01, -1.1872659e+00,
          -3.3039081e-01,  1.1499546e+00, -2.4678744e-04, -1.5465643e-01,
          -5.3651679e-01]],
 
        [[ 8.0435149e-02, -3.5867035e-01, -8.8504088e-01, -1.8140224e-01,
          -9.0008754e-01, -3.3983514e-01,  3.5120124e-01, -6.0242099e-01,
           3.3899057e-01]]], dtype=float

In [37]:
# serialize model to JSON
'''
model_json = model.to_json()
with open("./model_quantum-conv_quantum-fc_binary.json", "w") as json_file:
    json_file.write(model_json)
'''
# serialize weights to HDF5
model.save_weights("./model_quantum-conv_quantum-fc_binary_2.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
q_conv_layer_1.get_weights()

[array([[[ 0.44586676,  1.256093  ,  0.15643543,  0.26589197,
           0.8381965 ,  0.06237197,  0.5082177 ,  0.15269381,
           0.07892394]],
 
        [[-0.03475028,  1.0941952 , -0.03592378, -0.22752422,
           0.8826769 ,  0.13078535,  0.2010324 ,  0.22705875,
          -0.0801568 ]]], dtype=float32)]

In [ ]:
q_conv_layer_2.get_weights()

[array([[[ 0.24584407,  0.36597022, -0.07061452, -0.26450574,
           0.27333382, -0.1639238 ,  0.20467144,  0.66669524,
           0.10908085]],
 
        [[-0.08407965,  0.9870063 ,  0.2311346 , -0.25011623,
           0.5391546 ,  0.26538318,  0.32042062,  1.2632005 ,
           0.09457898]]], dtype=float32)]

In [ ]:
model_best_weights = model.get_weights()

In [ ]:
model_best_weights

[array([[[ 0.44586676,  1.256093  ,  0.15643543,  0.26589197,
           0.8381965 ,  0.06237197,  0.5082177 ,  0.15269381,
           0.07892394]],
 
        [[-0.03475028,  1.0941952 , -0.03592378, -0.22752422,
           0.8826769 ,  0.13078535,  0.2010324 ,  0.22705875,
          -0.0801568 ]]], dtype=float32),
 array([[[ 0.24584407,  0.36597022, -0.07061452, -0.26450574,
           0.27333382, -0.1639238 ,  0.20467144,  0.66669524,
           0.10908085]],
 
        [[-0.08407965,  0.9870063 ,  0.2311346 , -0.25011623,
           0.5391546 ,  0.26538318,  0.32042062,  1.2632005 ,
           0.09457898]]], dtype=float32),
 array([[[-0.2811071 ,  0.5625187 , -0.50280356,  0.44382098,
          -0.6297856 ,  1.1031843 ,  0.49210244, -0.14317082,
          -0.30502427]],
 
        [[-0.1502397 , -0.86425185, -0.21489263,  0.06380039,
          -0.4102494 ,  0.75493497,  0.68173337, -1.0992723 ,
          -0.06085886]]], dtype=float32),
 array([[1.0801506, 0.9991087]], dtype=float32)]

In [ ]:
predict_test = model.predict(X_test)

In [ ]:
'''
from keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

'''